# Large Sum 

Work out the first ten digits of the sum of the following one-hundred 50-digit numbers.

---


## Theory

## Implementation

In [2]:
fn num_additional_digits(num_lines: usize) -> usize {
    (num_lines as f64).log10().ceil() as usize
}

In [3]:
use std::fs::File;
use std::io::{BufReader, Error};
use std::io::prelude::*;

use std::collections::HashMap;

fn load_numbers(file_name: &String, num_lines: usize, num_digits:usize) -> Result<HashMap<usize, Vec<u8>>,Error > {
    
    // Open file
    let file = File::open(file_name)?;
    // Create buffer reader and content storage
    let mut buf_reader = BufReader::new(file);
    let mut contents = String::new();
    // Read file contents into string
    buf_reader.read_to_string(&mut contents)?;
    
    // Initialise vector to store output
    let mut output: HashMap<usize,Vec<u8>> = HashMap::with_capacity(num_digits);
    
    for key_val in 0..num_digits {
        output.insert(key_val, Vec::with_capacity(num_lines));
    }
    
    // Put data by position
    contents.lines().for_each(
        |line| line.char_indices().filter(|(index, ch)| index<&num_digits)
                .for_each(|ch| output.get_mut(&ch.0).unwrap().push(ch.1.to_digit(10).unwrap() as u8))
    );
    
    Ok(output)
}

In [4]:
// let test_val = load_numbers(String::from("p13_nums.txt"), 100, 13);
// println!("{:?}", &test_val.unwrap());

In [5]:
fn sum_columns_right_to_left(place_sums: &Vec<u16>, total_num_digits: usize) -> Vec<u8> {
    
    let mut digits_reversed: Vec<u8> = Vec::with_capacity(total_num_digits);
    
    let mut carry_over: u16 = 0;
    
    for curr_place_sum in place_sums.iter().rev() {
        let local_sum = curr_place_sum + carry_over;
        let digit = local_sum % 10;
        carry_over = local_sum / 10;
        digits_reversed.push(digit as u8);
    }
    
    while carry_over > 0 {
        let digit = carry_over % 10;
        digits_reversed.push(digit as u8);
        carry_over /= 10;
    }
    
    digits_reversed.reverse();
    
    digits_reversed
}

In [6]:
fn sum_large_numbers(file_name: &String, first_x_digits: usize, num_lines: usize) -> Result<Vec<u8>, Error> {
    
    let num_power_10 = num_additional_digits(num_lines);
    
    let num_digits = num_power_10 + first_x_digits;
    
    let input_data = load_numbers(file_name, num_lines, num_digits)?;
    
    let place_sums = (0..num_digits).map(|index| input_data.get(&index)
                                                            .unwrap()
                                                            .iter()
                                                            .fold(0_u16, 
                                                                |local_sum, val| 
                                                                local_sum + *val as u16))
                                        .collect::<Vec<u16>>();
    
    let total_sum = sum_columns_right_to_left(&place_sums, num_digits + num_power_10);
    
    Ok(total_sum)
}

In [7]:
fn convert_digit_array_to_string(digit_array: &Vec<u8>, first_n_digits: usize) -> String {
    digit_array[0..first_n_digits].iter().map(|digit| digit.to_string()).collect::<String>()
}

## Execution

In [8]:
let first_x_digits: usize = 10;
let num_numbers: usize = 100;
let input_filename: String = String::from("p13_nums.txt"); 

In [14]:
let digit_array = sum_large_numbers(&input_filename, first_x_digits, num_numbers)?;
println!("Array of digits {:?}", digit_array);
let soln = convert_digit_array_to_string(&digit_array, first_x_digits);
println!("First {} digits of the sum of {} 50-digit numbers = {:?}", first_x_digits, num_numbers, soln);

Array of digits [5, 5, 3, 7, 3, 7, 6, 2, 3, 0, 3, 8, 6, 0]
First 10 digits of the sum of 100 50-digit numbers = "5537376230"
